# МОДУЛЬ 4: Импорт, модули, пакеты, области видимости переменных

## 1. Области видимости

### 1.1. LEGB (Local, Enclosed, Global, Built-in)
**LEGB** описывает порядок, в котором Python ищет переменные:
1. **Local (локальная)**: Внутри текущей функции.
2. **Enclosed (охватывающая)**: Во внешней функции (если есть).
3. **Global (глобальная)**: На уровне модуля (файла с кодом).
4. **Built-in (встроенная)**: Встроенные имена Python (например, `len`, `print`).

Пример демонстрации LEGB:

In [1]:
x = 10  # Global

def outer_func():
    x = 20  # Enclosed
    
    def inner_func():
        x = 30  # Local
        print("Local x =", x)
    
    inner_func()
    print("Enclosed x =", x)

outer_func()
print("Global x =", x)
print("Built-in len =", len([1,2,3]))  # Built-in

Local x = 30
Enclosed x = 20
Global x = 10
Built-in len = 3


### 1.2. `global` и `nonlocal`
- **`global`**: Позволяет функции изменить значение глобальной переменной.
- **`nonlocal`**: Позволяет внутренней функции изменить переменную охватывающей функции (не глобальную).

In [2]:
# Пример использования global:
counter = 0

def increase_global():
    global counter
    counter += 1

increase_global()
print("Counter after increase_global:", counter)


Counter after increase_global: 1


In [3]:
# Пример использования nonlocal:
def outer():
    x = 5
    
    def inner():
        nonlocal x  # ссылаемся на переменную из outer()
        x += 10
        print("Inner x:", x)
    
    inner()
    print("Outer x:", x)

outer()

Inner x: 15
Outer x: 15


### 1.3. Scope vs Namespace
- **Scope (область видимости)** определяет, где доступна переменная.
- **Namespace (пространство имен)** — это коллекция имён и их значений (переменные, функции, классы и т.д.).

### 1.4. `globals()`, `dir()`, `locals()`
- **`globals()`**: возвращает словарь глобального пространства имен.
- **`locals()`**: возвращает словарь локального пространства имен.
- **`dir()`**: возвращает список имён текущего пространства (атрибутов модуля, объекта и т.п.).

In [4]:
def example_func():
    local_var = 123
    print("locals() inside function:", locals())

example_func()
print("globals() in module:", list(globals().keys())[:10], "...")  # Выведем часть ключей
print("dir() in module:", dir()[:10], "...")

locals() inside function: {'local_var': 123}
globals() in module: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh'] ...
dir() in module: ['In', 'NamespaceMagics', 'Out', '_', '__', '__K', '___', '__builtin__', '__builtins__', '__doc__'] ...


### 1.5. `if __name__ == '__main__'`
При запуске Python-файла напрямую, `__name__` в нем равно `'__main__'`. Это условие защищает код от выполнения при импортировании как модуля.

### 1.6. Процесс компиляции
В Python исходный код **компилируется** в байт-код (`.pyc`), который затем исполняется виртуальной машиной (PVM).

## 2. Дополнительно

### 2.1. «Моржовый оператор» (`:=`)
Позволяет присвоить переменной значение прямо внутри выражения.
```python
if (n := len([1,2,3])) > 2:
    print(n)
```

### 2.2. `func.__code__.co_varnames`
Хранит список имён локальных переменных функции. Пример:

In [5]:
def sample_func(a, b):
    x = a + b
    y = x * 2
    return y

print(sample_func.__code__.co_varnames)  # ('a', 'b', 'x', 'y')

('a', 'b', 'x', 'y')


## 3. Замыкания (Closures)

### 3.1. Определение, примеры
Замыкание — это функция, которая запоминает переменные из внешней области видимости, даже если сама внешняя функция уже завершила работу.

In [6]:
def make_multiplier(n):
    def multiplier(x):
        return x * n  # n из охватывающей области
    return multiplier

times3 = make_multiplier(3)
times5 = make_multiplier(5)

print(times3(10))  # 30
print(times5(10))  # 50

30
50


### 3.2. Зачем нужны
- Создание функций с состоянием.
- Реализация декораторов и колбэков.

### 3.3. Свободные переменные (free variables)
Переменные, используемые в замыкании, но не объявленные в самой внутренней функции.

### 3.4. Извлечение значений и имён
Можно использовать `__closure__` для анализа замыкания, но это довольно низкоуровневый подход.
```python
times3.__closure__  # Кортеж ячеек с сохранёнными переменными
```

## 4. Импорт

### 4.1. `__init__.py`, `__all__`, non-public
- `__init__.py` делает папку пакетом.
- `__all__` — список имён, импортируемых при `from module import *`.
- Имена, начинающиеся с `_`, считаются непубличными.

Пример структуры:
```
mypackage/
  __init__.py
  module_a.py
  module_b.py
```
```python
# mypackage/__init__.py
__all__ = ["module_a"]
```

### 4.2. `__import__`
Функция, позволяющая вручную импортировать модуль:
```python
mod = __import__("math")
print(mod.sqrt(16))
```

### 4.3. Абсолютные и относительные импорты
- **Абсолютные** указывают полный путь пакета.
- **Относительные** используют синтаксис точек (`.`, `..`) для указания пути относительно текущего модуля.

### 4.4. Где лежат используемые библиотеки? (`sys.modules`)
`sys.modules` — словарь всех уже загруженных модулей.
```python
import sys
print(sys.modules.keys())
```

## 5. Управление пакетами

### 5.1. APT / `python-apt`
- **APT** — менеджер пакетов в Linux.
- `python-apt` — модуль для взаимодействия с APT из Python.

### 5.2. `pip` и расположение пакетов
- **pip** — менеджер пакетов Python.
- Устанавливает пакеты глобально или в виртуальном окружении.
```bash
pip install requests  # Установка пакета
```

### 5.3. Работа с кэшем (`~/.cache/pip`, `pip cache purge`, `--no-cache-dir`)
- `~/.cache/pip` — директория кэша.
- `pip cache purge` — очистка кэша.
- `--no-cache-dir` — отключить использование кэша при установке пакета.

### 5.4. Связь `pip` и `gcc`
При установке пакетов с C/C++-кодом (например, `pandas`, `lxml`) может понадобиться **gcc** для их компиляции.

## 6. Интерактивные окружения

### 6.1. Google Colab
- В Colab можно запускать Python-код в облаке.
- Доступ к GPU/TPU, нет swap по умолчанию.
```bash
!python3 --version
```

## 7. Модуль `sys`

### 7.1. `sys.executable`
Путь к Python-интерпретатору.
```python
import sys
print(sys.executable)
```

### 7.2. `sys.argv`, `sys.exit()`
- `sys.argv` — список аргументов командной строки.
- `sys.exit()` — выход из программы.

### 7.3. `sys.path` и `sys.modules`
- `sys.path` — список путей, где Python ищет модули.
- `sys.modules` — словарь уже загруженных модулей.

### 7.4. `sys.version`, `sys.version_info`
- `sys.version` — строка с версией Python.
- `sys.version_info` — кортеж с деталями версии.

### 7.5. `sys.platform`
Строка с информацией о платформе (например, `win32`, `linux` или `darwin`).

### 7.6. `sys.getsizeof(object)` vs `object.__sizeof__()`
- `sys.getsizeof(object)` — возвращает полный размер объекта в байтах (включая накладные расходы).
- `object.__sizeof__()` — базовый метод, может не показывать полные накладные расходы.

### 7.7. `sys.getrefcount(object)`
Возвращает количество ссылок на объект. Может отличаться при работе в интерактивных средах (потому что сама функция создает временные ссылки).

## 8. Built-in модули и `importlib`

### 8.1. `builtins`, `__builtin__`, `__builtins__`
- `builtins` — основной модуль со встроенными функциями в Python 3.
- `__builtin__` — в Python 2, тоже модуль встроенных функций.
- `__builtins__` — словарь или ссылка на модуль встроенных объектов.

### 8.2. Просмотр версии модуля, `__name__`
- `module.__version__` (если предусмотрено) или `help(module)`.
- `__name__` показывает имя модуля (или `'__main__'` при прямом запуске).

### 8.3. `importlib.reload(...)`
- Позволяет перезагрузить уже импортированный модуль.
```python
import importlib
import math
importlib.reload(math)
```

### 8.4. Создание модулей на лету (`%%writefile`, `.import_module(...)`)
- В Jupyter Notebook можно использовать `%%writefile file.py`.
- `importlib.import_module(name)` динамически импортирует модуль по имени.

## 9. Структура проекта

### 9.1. `__init__.py`, `__all__`, `__doc__`, `__annotations__`
- `__init__.py` позволяет Python распознавать директорию как пакет.
- `__all__` описывает, какие имена экспортировать при `from package import *`.
- `__doc__` содержит строку документации модуля.
- `__annotations__` хранит аннотации типов.

### 9.2. Организация пакетов и подпакетов
Пакеты могут содержать подпакеты, которые сами имеют `__init__.py`. Глубокая иерархия пакетов позволяет удобно структурировать проект.